# Исследование предпочтений и активности пользователей сервиса Яндекс.Музыка

## Описание проекта  

В распоряжении имеется датасет с данными по использованию сервиса Яндекс.Музыка в Москве и Санкт-Петербурге. На основании представленных данных необходимо проверить три гипотезы (нестатистические) и сделать необходимые выводы.

## Цель исследования  

### Проверка трех гипотез: 

1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

## Описание данных

*Датасет yandex_music_project.csv*

*userID* — идентификатор пользователя;  
*Track* - название трека;  
*artist* - наименование испольнителя;  
*genre* - жанр музыки;  
*City* - город пользователя;  
*time* - время суток;  
*Day* - день недели.  

## Содержание:  

1 Обзор данных  
2 Предобработка данных  
2.1 Стиль заголовков  
2.2 Пропуски значений  
2.3 Дубликаты  
2.4 Выводы   
3 Проверка гипотез  
3.1 Сравнение поведения пользователей двух столиц  
3.2 Музыка в начале и в конце недели  
3.3 Жанровые предпочтения в Москве и Петербурге  
4 Итоги исследования  

## 1 Обзор данных

Загружаем библиотеку pandas:

In [1]:
import pandas as pd

Открываем файл и сохраняем его в переменной `df`:

In [2]:
df = pd.read_csv('/datasets/yandex_music_project.csv')

Выводим на экран первые десять строк таблицы:

In [3]:
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


Теперь посмотрим общую информацию о таблице:

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.

В названиях колонок видны три нарушения стиля:
1. Строчные буквы сочетаются с прописными.
2. Встречаются пробелы.
3. Название, состоящее из нескольких слов, соединяется в одно без использования символа нижнего подчеркивания.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

**Выводы**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## 2 Предобработка данных

### 2.1 Стиль заголовков

Для начала выведем названия столбцов:

In [5]:
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Теперь приведем названия столбцов в соответствии с правилами хорошего стиля (уберем пробелы и все символы сделаем строчными), а заодно, изменим названия столбца c днями недели *Day* на более точное - *weekday*:

In [6]:
df = df.rename(columns={'  userID': 'user_id', 'Track': 'track', '  City  ': 'city', 'Day': 'weekday'})

Проверим результат переименования:

In [7]:
print(df.columns) #  проверка результатов - перечень названий столбцов

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'weekday'], dtype='object')


### 2.2 Пропуски значений

Посчитаем сумму пропущенных значений по столбцам:

In [8]:
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
weekday       0
dtype: int64

Не все пропущенные значения влияют на исследование. Так, в `track` и `artist` пропуски будут не важны для данной работы. Достаточно заменить их явными обозначениями.

Но пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. На практике было бы правильно установить причину пропусков и восстановить данные. Поскольку такой возможности в учебном проекте нет - заполним эти пропуски явными обозначениями и оценим, насколько они повредят расчётам. 

Заменим пропущенные значения в столбцах `track`, `artist` и `genre` на строку `'unknown'`. Для этого создадим список `columns_to_replace`, переберем его элементы циклом `for` и для каждого столбца выполним замену пропущенных значений:

In [9]:
columns_to_replace = ['track', 'artist', 'genre']
for element in columns_to_replace:
    df[element] = df[element].fillna('unknown') #  перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'

Проверим результат:

In [10]:
df.isna().sum()  # подсчёт пропусков

user_id    0
track      0
artist     0
genre      0
city       0
time       0
weekday    0
dtype: int64

### 2.3 Дубликаты
Посчитаем явные дубликаты в таблице:

In [11]:
print('Явных дубликатов:', df.duplicated().sum())

Явных дубликатов: 3826


Удалим их:

In [12]:
df = df.drop_duplicates().reset_index(drop=True) #  удаляем явные дубликаты и обновляем значения индексов
print('Явных дубликатов:', df.duplicated().sum()) #  контролируем результат

Явных дубликатов: 0


Теперь проверим колонку `genre` на наличие неявных дубликатов, для этого выведем уникальные значения с сортировкой по убыванию:

In [13]:
print(df['genre'].sort_values().unique())

['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'alternativepunk' 'ambient' 'americana' 'animated' 'anime' 'arabesk'
 'arabic' 'arena' 'argentinetango' 'art' 'audiobook' 'author' 'avantgarde'
 'axé' 'baile' 'balkan' 'beats' 'bigroom' 'black' 'bluegrass' 'blues'
 'bollywood' 'bossa' 'brazilian' 'breakbeat' 'breaks' 'broadway'
 'cantautori' 'cantopop' 'canzone' 'caribbean' 'caucasian' 'celtic'
 'chamber' 'chanson' 'children' 'chill' 'chinese' 'choral' 'christian'
 'christmas' 'classical' 'classicmetal' 'club' 'colombian' 'comedy'
 'conjazz' 'contemporary' 'country' 'cuban' 'dance' 'dancehall' 'dancepop'
 'dark' 'death' 'deep' 'deutschrock' 'deutschspr' 'dirty' 'disco' 'dnb'
 'documentary' 'downbeat' 'downtempo' 'drum' 'dub' 'dubstep' 'eastern'
 'easy' 'electronic' 'electropop' 'emo' 'entehno' 'epicmetal' 'estrada'
 'ethnic' 'eurofolk' 'european' 'experimental' 'extrememetal' 'fado'
 'fairytail' 'film' 'fitness' 'flamenco' 'folk' 'folklore' 'folkmetal'
 'folkrock' 

Действительно, название жанра `hiphop` представлено еще в трех вариантах:

* *hip*,
* *hop*,
* *hip-hop*.

Название жанра `electronic` представлено еще на русском языке:

* *электроника*

Чтобы упорядочить таблицу, напишем функцию `replace_wrong_genres()` с двумя параметрами: 
* `wrong_value` — некорректное значение (список значений),
* `correct_value` — корректное значение

Функция должна исправить колонку `genre` в таблице `df`: заменить каждое значение `wrong_value` на значение из `correct_value`:

In [14]:
def replace_wrong_genres(wrong_value, correct_value): #  функция для замены неявных дубликатов
    for i in wrong_value:
        df['genre'] = df['genre'].replace(wrong_value, correct_value)

Вызываем функцию и передаем ей аргументы для устранения неявных дубликатов:

In [15]:
replace_wrong_genres(['hip', 'hop', 'hip-hop'], 'hiphop')
replace_wrong_genres('электроника', 'electronic')

Контролируем результат:

In [16]:
print(df['genre'].sort_values().unique()) #  выводим еще раз уникальные значения жанров

['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'alternativepunk' 'ambient' 'americana' 'animated' 'anime' 'arabesk'
 'arabic' 'arena' 'argentinetango' 'art' 'audiobook' 'author' 'avantgarde'
 'axé' 'baile' 'balkan' 'beats' 'bigroom' 'black' 'bluegrass' 'blues'
 'bollywood' 'bossa' 'brazilian' 'breakbeat' 'breaks' 'broadway'
 'cantautori' 'cantopop' 'canzone' 'caribbean' 'caucasian' 'celtic'
 'chamber' 'chanson' 'children' 'chill' 'chinese' 'choral' 'christian'
 'christmas' 'classical' 'classicmetal' 'club' 'colombian' 'comedy'
 'conjazz' 'contemporary' 'country' 'cuban' 'dance' 'dancehall' 'dancepop'
 'dark' 'death' 'deep' 'deutschrock' 'deutschspr' 'dirty' 'disco' 'dnb'
 'documentary' 'downbeat' 'downtempo' 'drum' 'dub' 'dubstep' 'eastern'
 'easy' 'electronic' 'electropop' 'emo' 'entehno' 'epicmetal' 'estrada'
 'ethnic' 'eurofolk' 'european' 'experimental' 'extrememetal' 'fado'
 'fairytail' 'film' 'fitness' 'flamenco' 'folk' 'folklore' 'folkmetal'
 'folkrock' 

## 2.4 Выводы

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Исправилены заголовки, чтобы упростить работу с таблицей. Без дубликатов исследование станет более точным.

Пропущенные значения заменили на `'unknown'`. Ещё предстоит увидеть, не повредят ли исследованию пропуски в колонке `genre`.

Теперь можно перейти к проверке гипотез. 

## 3 Проверка гипотез

### 3.1 Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга;  
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.


Оценим активность пользователей в каждом городе. Сгруппируем данные по городу и посчитаем прослушивания в каждой группе:

In [17]:
df.groupby('city')['track'].count()

city
Moscow              42741
Saint-Petersburg    18512
Name: track, dtype: int64

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку, просто самих пользователей в Москве больше.

Теперь сгруппируем данные по дням недели и подсчитаем прослушивания:

In [18]:
print(df.groupby('weekday')['track'].count())

weekday
Friday       21840
Monday       21354
Wednesday    18059
Name: track, dtype: int64


В наличии имеются данные по трем дням недели (понедельник, среда и пятница). В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Напишем функцию `number_tracks()`, которая объединит два предыдущих расчета и выведет прослушивания для заданного дня и города. Ей понадобятся два параметра:
* день недели,
* название города.

В функции сохраним в переменную строки исходной таблицы, у которых значение:
  * в колонке `weekday` равно параметру `weekday`,
  * в колонке `city` равно параметру `city`.

Для этого применим последовательную фильтрацию с логической индексацией, затем посчитаем значения в столбце `user_id` получившейся таблицы. Результат сохраним в новую переменную `track_list_count`:

In [19]:
def number_tracks(weekday, city):
    track_list = df[(df['weekday'] == weekday) & (df['city'] == city)]
    track_list_count = track_list['user_id'].count()
    return track_list_count

Теперь вызовем `number_tracks()` шесть раз, меняя значение параметров — так, чтобы получить данные для каждого города в каждый из трёх дней:

In [20]:
number_tracks('Monday', 'Moscow') #  количество прослушиваний в Москве по понедельникам

15740

In [21]:
number_tracks('Monday', 'Saint-Petersburg')  # количество прослушиваний в Санкт-Петербурге по понедельникам

5614

In [22]:
number_tracks('Wednesday', 'Moscow') #  количество прослушиваний в Москве по средам

11056

In [23]:
number_tracks('Wednesday', 'Saint-Petersburg') #  количество прослушиваний в Санкт-Петербурге по средам

7003

In [24]:
number_tracks('Friday', 'Moscow') #  количество прослушиваний в Москве по пятницам

15945

In [25]:
number_tracks('Friday', 'Saint-Petersburg') #  количество прослушиваний в Санкт-Петербурге по пятницам

5895

Создадим c помощью конструктора `pd.DataFrame` таблицу, где
* названия колонок — дни недели `['city', 'monday', 'wednesday', 'friday']`;
* данные — результаты, которые мы получили с помощью функции`number_tracks`:

In [26]:
number_tracks = [
    ['Moscow', '15740', '11056', '15945'],
    ['saint-Petersburg', '5614', '7003', '5895']
]
columns = ['city', 'monday', 'wednesday', 'friday']
info = pd.DataFrame(data = number_tracks, columns = columns)
info

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,saint-Petersburg,5614,7003,5895


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Следовательно, данные говорят в пользу первой гипотезы.

### 3.2 Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Сохраним таблицы с данными в две переменные:
* по Москве — в `moscow_general`;
* по Санкт-Петербургу — в `spb_general`.

In [27]:
moscow_general = df[df['city'] == 'Moscow']

# получение таблицы moscow_general из тех строк таблицы df, 
# для которых значение в столбце 'city' равно 'Moscow'

In [28]:
spb_general = df[df['city'] == 'Saint-Petersburg']

# получение таблицы spb_general из тех строк таблицы df,
# для которых значение в столбце 'city' равно 'Saint-Petersburg'

Теперь создадим функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm'.

Функция должна вернуть информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [29]:
'''Объявляем функцию genre_weekday() с параметрами table, day, time1, time2,
   которая возвращает информацию о самых популярных жанрах в указанный день в
   заданное время:
'''
def genre_weekday(df, weekday, time1, time2):
    
     
    '''  В переменную genre_df сохраняются те строки переданного датафрейма table, для
     которых одновременно:
     - значение в столбце day равно значению аргумента day
     - значение в столбце time больше значения аргумента time1
     - значение в столбце time меньше значения аргумента time2
     Используем последовательную фильтрацию с помощью логической индексации.
    '''
    genre_df = df[(df['weekday'] == weekday) &
                  (df['time'] > time1) &
                  (df['time'] < time2)]
    
    
    '''Группируем датафрейм genre_df по столбцу genre, берем один из его
       столбцов и считаем методом count() количество записей для каждого из
       присутствующих жанров, получившийся Series записываем в переменную
       genre_df_count
    '''
    genre_df_count = genre_df.groupby('genre')['user_id'].count()
    
    
    '''Cортируем genre_df_count по убыванию встречаемости и сохраняем
       в переменную genre_df_sorted
    '''
    genre_df_sorted = genre_df_count.sort_values(ascending=False)
    
    
    '''Возвращаем Series из 10 первых значений genre_df_sorted, это будут топ-10
       популярных жанров (в указанный день, в заданное время)
    '''
    return genre_df_sorted.head(10)

Cравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [30]:
genre_weekday(moscow_general, 'Monday', '07:00', '11:00') #  вызов функции для утра понедельника в Москве

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: user_id, dtype: int64

In [31]:
genre_weekday(spb_general, 'Monday', '07:00', '11:00') #  вызов функции для утра понедельника в Петербурге

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: user_id, dtype: int64

In [32]:
genre_weekday(moscow_general, 'Friday', '17:00', '23:00') #  вызов функции для вечера пятницы в Москве

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: user_id, dtype: int64

In [33]:
genre_weekday(spb_general, 'Friday', '17:00', '23:00') #  вызов функции для вечера пятницы в Петербурге

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: user_id, dtype: int64

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### 3.3 Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппируем таблицу `moscow_general` по жанру и посчитаем прослушивания треков каждого жанра методом `count()`. Затем отсортируем результат в порядке убывания, сохраним его в таблице `moscow_genres` и выведем первые 10 строк:

In [34]:
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending=False)
moscow_genres.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Теперь повторим то же для Петербурга, сгруппируем таблицу `spb_general` по жанру, посчитаем прослушивания треков каждого жанра,  отсортируем результат в порядке убывания, сохраним в таблице `spb_genres` и выведем первые 10 строк:


In [35]:
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending=False)
spb_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1737
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## 4 Итоги исследования

**В рамках данного исследования были проверены три нестатистические гипотезы. Установлено следующее:**

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.

**Примечание:**
В данном проекте не требовалось использовать проверки статистических гипотез, хотя в исследованиях обычно такие проверки проводят.  Кроме того, на основе данных одного сервиса не всегда можно сделать вывод обо всех жителях города.